# Abordagem 3

Usando a abordagem 3 para gerar templates com foco em templates positivos e negativos. Uma possível aplicação seria testar a capacidade linguística *Vocabullary* com o teste **MFT**.

As etapas desta abordagem são:

1. Quebrar a instância em sentenças
2. Classificar as sentenças usando o *Oráculo*
3. Filtrar as sentenças classificadas de forma unânime
4. Filtrar as sentenças com alta confiança nas predições
5. Rankear as palavras de cada sentença
6. Filtrar sentenças com palavras relevantes (verbos ou adjetivos) bem rankeadas
7. Filtrar sentenças com alta confiança na predição das palavras relevantes 
8. Substituir as palavras relevantes por máscaras

In [1]:
%config Completer.use_jedi = False
import sys
sys.path.append('../')

## Carregando o dataset, o modelo alvo e os modelos auxiliares

In [2]:
import pandas as pd
pd.set_option('display.max_colwidth', None)

imdb_df = pd.read_csv('./data/imdb_sampled/data-100samples.csv')
imdb_df.head(5)

,label,text,words
0,1,"Christian Duguay directed this tidy little espionage thriller early in his career. It plays on TV pretty regularly, albeit with some terrific scenes of violence and sex unfortunately trimmed. I finally got around to seeing the theatrical version on a $3 tape from the local video store. Naval officer Aidan Quinn is recruited to impersonate the notorious Carlos the Jackal, and gets a little too caught up in the role. Donald Sutherland Ben Kingsley play Quinn's superiors, with Sutherland a true zealot and Kingsley as the more level-headed one. The first half of this fun flick shows Quinn being trained and indoctrinated. The second half has him out in the field, making love to the Jackal's woman and shooting it out with sundry enemies. The idea is to make the Jackal look like a turncoat to the Russians, and let them take care of the world's most notorious assassin. Things don't exactly play out as planned. At times, I almost expected the cast to break out laughing at some of the corny dialogue, but they all play it very straight. In the end, this is one terrific little thriller that deserves your attention. The Jackal's former mistress teaching the highly proper and very married Quinn to rough her up, lick blood from her face, and then go down on her, alone is worth the price of admission.",227
1,1,"New Yorkers contemporaneous with this film will recall how reflective of its time it is and how well cast and crew captured America, New York City of that era.<br /><br />Norman Wexler's script delineates the different worlds the various sub groupings live in and Avildsen's direction brings out phenomenal performances all around. Peter Boyle's prodigious talent is on display as never before nor since. Clearly it is the best character portrayal the always likable Dennis Patrick ever accomplished.<br /><br />What I will always remember about JOE is the feeling of having been in a virtual state of shock coming out of the theater. Knowing that what the screen portrayed was seething under the surface in neighborhoods throughout the five boroughs of the City of New York.<br /><br />This film needs to be remembered.",133
2,0,"I love oddball animation, I love a lot of Asian films, but I didn't love this particular product of Japan. The Fuccons are supposedly an American family (they're all mannequins) who have moved to Japan, and they're somewhat a 50's sitcom type family, with slightly more modern sensibilities at times. The DVD features several very short episodes (like less than 5 minutes each?) and I did not find it to be either funny or entertaining, not even in a weird way. I'm not sure what the appeal is of this. I did pick up on some satire here and there, gosh, who wouldn't, but satire is usually somewhat humorous, isn't it? And nothing I saw or heard rated even a little smirk. I picked this up used and it certainly SOUNDED appealing, but I guess either I'm missing the point or it's just plain LAME. The box even says it's Fuccon hilarious, right there on the front, but I beg to differ. 2 out of 10.",166
3,1,"I have seen this film probably a dozen times since it was originally released theatrically. Anyone who calls this movie trash or horrible just doesn't understand action films or recognize a good one. Perhaps to some the incidents and outcomes may seem far fetched, but in my opinion screenwriter Shane Black ( Lethal Weapon/ Kiss Kiss Bang Bang) crafted one of the most well thought out action adventures you will ever come across. Over the top or not this film flows like clockwork and the action just keeps coming. The final action sequence is one of the best I have ever seen in any film. The cast in this film crackles. Genna Davis gave a tremendous performance and its a damn shame there was never a ""LKG"" sequel. Samuel L. Jackson is hilarious as her sidekick Mitch a down on his luck private eye trying to help her discover her lost past and make a few bucks. If Baffles me how anyone could not like this film. I

In [3]:
import re
import numpy as np
from torch.nn.functional import softmax
from transformers import AutoTokenizer, AutoModelForSequenceClassification

def pre_proccess(text):
    text = text.lower()
    text = re.sub('["\',!-.:-@0-9/]()', ' ', text)
    return text

# Wrapper to adapt output format
class SentimentAnalisysModelWrapper:
    def __init__(self, model, tokenizer):
        self.model = model
        self.tokenizer = tokenizer
        
    def __predict(self, text_input):
        text_preprocessed = pre_proccess(text_input)
        tokenized = self.tokenizer(text_preprocessed, padding=True, truncation=True, max_length=512, 
                                    add_special_tokens = True, return_tensors="pt")
        
        tensor_logits = self.model(**tokenized)
        prob = softmax(tensor_logits[0]).detach().numpy()
        pred = np.argmax(prob)
        
        return pred, prob
    
    def predict_label(self, text_inputs):
        return self.predict(text_inputs)[0]
        
    def predict_proba(self, text_inputs):
        return self.predict(text_inputs)[1]
        
    def predict(self, text_inputs):
        if isinstance(text_inputs, str):
            text_inputs = [text_inputs]
        
        preds = []
        probs = []

        for text_input in text_inputs:
            pred, prob = self.__predict(text_input)
            preds.append(pred)
            probs.append(prob[0])

        return np.array(preds), np.array(probs) # ([0, 1], [[0.99, 0.01], [0.03, 0.97]])

# Auxiliar function to load and wrap a model from Hugging Face
def load_model(model_name):
    print(f'Loading model {model_name}...')
    model = AutoModelForSequenceClassification.from_pretrained(model_name)
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    
    return SentimentAnalisysModelWrapper(model, tokenizer)

# Hugging Face hosted model names 
imdb_models = {
    'bert': 'textattack/bert-base-uncased-imdb', 
    'albert': 'textattack/albert-base-v2-imdb', 
    'distilbert': 'textattack/distilbert-base-uncased-imdb', 
    'roberta': 'textattack/roberta-base-imdb', 
    'xlnet': 'textattack/xlnet-base-cased-imdb'
}

In [4]:
m1 = load_model(imdb_models['albert'])
m2 = load_model(imdb_models['distilbert'])
m3 = load_model(imdb_models['roberta'])
m4 = load_model(imdb_models['xlnet'])

# Models to be used as oracle
models = [m1, m2, m3, m4]
# Target model
model = load_model(imdb_models['bert'])

Loading model textattack/albert-base-v2-imdb...
Loading model textattack/distilbert-base-uncased-imdb...
Loading model textattack/roberta-base-imdb...


Some weights of the model checkpoint at textattack/roberta-base-imdb were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Loading model textattack/xlnet-base-cased-imdb...
Loading model textattack/bert-base-uncased-imdb...


# Gerando os templates
O método de rankeamento das palavras usado no PosNegTemplateGenerator é o Replace-1 Score

In [5]:
from template_generator.tasks.sentiment_analisys import PosNegTemplateGeneratorApp3

tg = PosNegTemplateGeneratorApp3(model, models)

### Número inicial de instâncias: 5

In [6]:
# Sampling instances
np.random.seed(220)
n_instances = 5
df_sampled = imdb_df.sample(n_instances)

instances = [x for x in df_sampled['text'].values]

In [7]:
templates = tg.generate_templates(instances, ranked_words_count=4, min_classification_score=0.8)

Converting texts to sentences...
:: 37 sentences were generated.
Predicting inputs...


<ipython-input-3-3748665271fa>:23: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  prob = softmax(tensor_logits[0]).detach().numpy()


:: Sentence predictions done.
Filtering instances classified unanimously...
:: 28 sentences remaining.
Filtering instances by classification score greater than 0.8
:: 27 sentences remaining.
Ranking words using Replace-1 Score...
:: Word ranking done.
Filtering instances by relevant words...
:: 10 sentences remaining.
Filtering instances by relevant words classification score greater than 0.8
:: 1 sentences remaining.


#### Tempo de execução para 5 instâncias: 1m 19.9s

In [8]:
df = tg.to_dataframe()
df

,label,original_text,masked_text,template_text
0,1,"Just forget the itty-bitty disappointments, like the fact that there were only adults in this movie based on a pup's point of view, because that's just 0.5% or less of the movie's wonderful effect on the viewer.","Just {mask} the itty-bitty disappointments , like the fact that there were only adults in this movie based on a pup 's point of view , because that 's just 0.5 % or less of the movie 's {mask} effect on the viewer .","Just {neg_verb} the itty-bitty disappointments , like the fact that there were only adults in this movie based on a pup 's point of view , because that 's just 0.5 % or less of the movie 's {pos_adj} effect on the viewer ."


In [9]:
tg.lexicons

{'pos_verb': [],
 'neg_verb': ['forget'],
 'pos_adj': ['wonderful'],
 'neg_adj': []}

### Número inicial de instâncias: 100

In [10]:
# Using all 100 instances
instances = [x for x in imdb_df['text'].values]

In [11]:
tg = PosNegTemplateGeneratorApp3(model, models)
templates = tg.generate_templates(instances, ranked_words_count=4, min_classification_score=0.8)

Converting texts to sentences...
:: 793 sentences were generated.
Predicting inputs...


<ipython-input-3-3748665271fa>:23: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  prob = softmax(tensor_logits[0]).detach().numpy()


:: Sentence predictions done.
Filtering instances classified unanimously...
:: 498 sentences remaining.
Filtering instances by classification score greater than 0.8
:: 444 sentences remaining.
Ranking words using Replace-1 Score...
:: Word ranking done.
Filtering instances by relevant words...
:: 175 sentences remaining.
Filtering instances by relevant words classification score greater than 0.8
:: 18 sentences remaining.


#### Tempo de execução para 100 instâncias: 21m 55.9s

In [12]:
df = tg.to_dataframe()
df

,label,original_text,masked_text,template_text
0,0,"Directed, co-written, co-produced and co-edited with dumbfounding maladroitness by Melanie Anne Phillips, acted with dismaying flatness by a rank no-name cast, further marred by lethargic pacing, a drably meandering narrative, murky, under-lit, eye-straining cinematography, a shivery, redundantly thudding pseudo-John Carpenter synthesizer score, and a cruddy, herky-jerky stop motion animation wormoid thingie that's only quickly glimpsed at the very end of the movie, this extremely clunky, amateurish and hence quite delectably dreadful would-be scarefest commits all the necessary bad film missteps to qualify as a real four-star stinkeroonie.","Directed , co-written , co-produced and co-edited with {mask} maladroitness by Melanie Anne Phillips , acted with dismaying flatness by a rank no-name cast , further marred by lethargic pacing , a drably meandering narrative , murky , under-lit , eye-straining cinematography , a shivery , redundantly thudding pseudo-John Carpenter synthesizer score , and a cruddy , herky-jerky stop motion animation wormoid thingie that 's only quickly glimpsed at the very end of the movie , this extremely clunky , {mask} and hence quite delectably dreadful would-be scarefest commits all the necessary bad film missteps to qualify as a real four-star stinkeroonie .","Directed , co-written , co-produced and co-edited with {neg_verb} maladroitness by Melanie Anne Phillips , acted with dismaying flatness by a rank no-name cast , further marred by lethargic pacing , a drably meandering narrative , murky , under-lit , eye-straining cinematography , a shivery , redundantly thudding pseudo-John Carpenter synthesizer score , and a cruddy , herky-jerky stop motion animation wormoid thingie that 's only quickly glimpsed at the very end of the movie , this extremely clunky , {neg_adj} and hence quite delectably dreadful would-be scarefest commits all the necessary bad film missteps to qualify as a real four-star stinkeroonie ."
1,1,"Just forget the itty-bitty disappointments, like the fact that there were only adults in this movie based on a pup's point of view, because that's just 0.5% or less of the movie's wonderful effect on the viewer.","Just {mask} the itty-bitty disappointments , like the fact that there were only adults in this movie based on a pup 's point of view , because that 's just 0.5 % or less of the movie 's {mask} effect on the viewer .","Just {neg_verb} the itty-bitty disappointments , like the fact that there were only adults in this movie based on a pup 's point of view , because that 's just 0.5 % or less of the movie 's {pos_adj} effect on the viewer ."
2,1,But not only are messages great- its hilarious!,But not only are messages {mask} its {mask} !,But not only are messages {pos_adj} its {pos_adj} !
3,1,It carries the unique bond of a friendship between a teenager and a dolphin.,It {mask} the {mask} bond of a friendship between a teenager and a dolphin .,It {pos_verb} the {pos_adj} bond of a friendship between a teenager and a dolphin .
4,0,"<br /><br />I would have rated this one star (awful), but the music wasn't that bad, and I did like the premise.","< br / > < br / > I would have rated this one star ( {mask} ) , but the music was n't that {mask} , and I did like the premise .","< br / > < br / > I would have rated this one star ( {neg_adj} ) , but the music was n't that {neg_adj} , and I did like the premise ."
5,1,"Good directing, good script, worth renting.","{mask} directing , {mask} script , worth renting .","{pos_adj} directing , {pos_adj} script , worth renting ."
6,0,"The basic premise seems okay, but the plot was ridiculously involved and tortuous, and runs out half way through.","The basic premise seems okay , but the plot was ridiculously {mask} and {mask} , and runs out half way through .","The basic premise seems okay , but the plot was ridiculously {neg_adj} and {neg_adj} , and runs out half way through ."
7,1,It is hilarious

In [13]:
tg.lexicons

{'pos_verb': ['love', 'carries', 'sit'],
 'neg_verb': ['should',
  'dumbfounding',
  'shambolic',
  'replaced',
  'forget',
  'had',
  'homophobic'],
 'pos_adj': ['good',
  'well-developed.Still',
  'hilarious',
  'great-',
  'well-acted',
  'moster-run-amok',
  'solid',
  'wonderful',
  'Good',
  'unique'],
 'neg_adj': ['ridiculous',
  'amateurish',
  'stereotypical',
  'dull',
  'poor',
  'silly',
  'absurd',
  'involved',
  'boring',
  'awful',
  'bad',
  'sick',
  'tortuous']}

# Usando os templates gerados pelo TemplateGenerator no CheckList

In [14]:
from checklist.editor import Editor
from checklist.test_suite import TestSuite
from checklist.test_types import MFT

In [15]:
lexicons = tg.lexicons
templates = tg.template_texts
masked = tg.masked_texts
labels = [sent.prediction.label for sent in tg.sentences]

editor = Editor()
editor.add_lexicon('pos_verb', lexicons['pos_verb'])
editor.add_lexicon('neg_verb', lexicons['neg_verb'])
editor.add_lexicon('pos_adj', lexicons['pos_adj'])
editor.add_lexicon('neg_adj', lexicons['neg_adj'])

suite = TestSuite()

In [16]:
for template, label, i in zip(templates, labels, range(len(templates))):
    t = editor.template(template, remove_duplicates=True, labels=int(label))

    suite.add(MFT(
        data=t.data,
        labels=label,
        capability="Vocabullary", 
        name=f"Test: MFT with vocabullary - template{i+1}",
        description="Checking if the model can handle vocabullary"))

In [17]:
suite.run(model.predict, overwrite=True)
suite.save('./suites/posneg-approach3.suite')

Running Test: MFT with vocabullary - template1
Predicting 91 examples


<ipython-input-3-3748665271fa>:23: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  prob = softmax(tensor_logits[0]).detach().numpy()


Running Test: MFT with vocabullary - template2
Predicting 70 examples
Running Test: MFT with vocabullary - template3
Predicting 10 examples
Running Test: MFT with vocabullary - template4
Predicting 30 examples
Running Test: MFT with vocabullary - template5
Predicting 13 examples
Running Test: MFT with vocabullary - template6
Predicting 10 examples
Running Test: MFT with vocabullary - template7
Predicting 13 examples
Running Test: MFT with vocabullary - template8
Predicting 130 examples
Running Test: MFT with vocabullary - template9
Predicting 130 examples
Running Test: MFT with vocabullary - template10
Predicting 91 examples
Running Test: MFT with vocabullary - template11
Predicting 21 examples
Running Test: MFT with vocabullary - template12
Predicting 130 examples
Running Test: MFT with vocabullary - template13
Predicting 130 examples
Running Test: MFT with vocabullary - template14
Predicting 30 examples
Running Test: MFT with vocabullary - template15
Predicting 13 examples
Running Te

# Carregando suite de teste

In [1]:
from checklist.test_suite import TestSuite
suite = TestSuite.from_file('./suites/posneg-approach3.suite')

suite.visual_summary_table()

Please wait as we prepare the table data...


SuiteSummarizer(stats={'npassed': 0, 'nfailed': 0, 'nfiltered': 0}, test_infos=[{'name': 'Test: MFT with vocab…